# SMAs

In [ ]:
Def SMAs_Function(combined_values, short_window = 20, long_window = 100)
    ### Step 1: Import the dataset into a Pandas DataFrame.
    # Filter the date index and close columns
    signals_df = combined_values.loc[:, ["Close"] ["value"]]

    # Use the pct_change function to generate  returns from close prices
    signals_df["Actual Returns"] = signals_df["Close"].pct_change()

    # Drop all NaN values from the DataFrame
    signals_df = signals_df.dropna()

    # Review the DataFrame
    # display(signals_df.head())
    # display(signals_df.tail())
    
    ## Step 2: Generate trading signals using short- and long-window SMA values. 
    # Set the short window and long window
    #short_window = 20
    #long_window = 100

    # Generate the fast and slow simple moving averages (4 and 100 days, respectively)
    signals_df['SMA_Fast'] = signals_df['Close'].rolling(window=short_window).mean()
    signals_df['SMA_Slow'] = signals_df['Close'].rolling(window=long_window).mean()

    signals_df = signals_df.dropna()

    # Review the DataFrame
    display(signals_df.head())
    display(signals_df.tail())
    
    # Initialize the new Signal column
    signals_df['Signal'] = 0.0

    # When Actual Returns are greater than or equal to 0, generate signal to buy stock long
    signals_df.loc[(signals_df['Actual Returns'] >= 0), 'Signal'] = 1

    # When Actual Returns are less than 0, generate signal to sell stock short
    signals_df.loc[(signals_df['Actual Returns'] < 0), 'Signal'] = -1

    # Review the DataFrame
    display(signals_df.head())
    display(signals_df.tail())
    
    display(signals_df['Signal'].value_counts())
    
    # Calculate the strategy returns and add them to the signals_df DataFrame
    signals_df['Strategy Returns'] = signals_df['Actual Returns'] * signals_df['Signal'].shift()

    # Review the DataFrame
    display(signals_df.head())
    display(signals_df.tail())
    
    # Plot Strategy Returns to examine performance
    (1 + signals_df['Strategy Returns']).cumprod().plot()
    
    ### Step 3: Split the data into training and testing datasets.
    
    # Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
    X = signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()

    # Review the DataFrame
    display(X.head())
    
    # Create the target set selecting the Signal column and assiging it to y
    y = signals_df['Signal']

    # Review the value counts
    y.value_counts()
    
    # Select the start of the training period
    training_begin = X.index.min()

    # Display the training begin date
    display(training_begin)
    
    # Select the ending period for the training data with an offset of 3 months
    training_end = X.index.min() + DateOffset(months=3)

    # Display the training end date
    display(training_end)
    
    # Generate the X_train and y_train DataFrames
    X_train = X.loc[training_begin:training_end]
    y_train = y.loc[training_begin:training_end]

    # Review the X_train DataFrame
    X_train.head()
    
    # Generate the X_test and y_test DataFrames
    X_test = X.loc[training_end+DateOffset(hours=1):]
    y_test = y.loc[training_end+DateOffset(hours=1):]

    # Review the X_test DataFrame
    X_train.head()
    
    # Scale the features DataFrames

    # Create a StandardScaler instance
    scaler = StandardScaler()

    # Apply the scaler model to fit the X-train data
    X_scaler = scaler.fit(X_train)

    # Transform the X_train and X_test DataFrames using the X_scaler
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    
    return  X_train_scaled, y_train, y_test